In [14]:
%load_ext autoreload
%autoreload parse

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
import pgf
import main
import parse
import analysis

In [11]:
gr = pgf.readPGF('../data/TranslateEngSwe.pgf')
eng = gr.languages['TranslateEng']
swe = gr.languages['TranslateSwe']
p, ex = next(eng.parse('the horse likes to eat the hay which we all had selected'))

In [94]:
p, ex = next(eng.parse('I look at you'))
swe.linearize(pgf.readExpr('think_VS'))

'tänka'

In [3]:
f, args = ex.unpack()
f, args = args[1].unpack()
f, args = args[0].unpack()
predvp = args[2]

In [6]:
labels['which_IQuant']

['head']

In [5]:
from itertools import takewhile
from collections import defaultdict
with open('../data/Lang.labels') as f:
    rows = [l.strip().split() for l in f if l.strip() != '']
    labels = defaultdict(lambda: ['head'])
    for row in rows:
        fun, *rest = row
        args = takewhile(lambda w: w[0:2] != '--', rest)
        labels[fun] = list(args)
labels['Utt']
#[l for l in labels if len(l) == 0]

['head']

In [38]:
from itertools import chain
def find_heads(expression, prev_heads = [], label='root'):
    fun, arguments = expression.unpack()
    arg_labels = labels[fun]
    headi = arg_labels.index('head')
    if (len(arguments) <= headi):
        return [(fun, prev_heads, label)], fun
    else:
        out, head = gen_trigrams(arguments[headi], prev_heads, label)
        for i, arg in enumerate(arguments):
            if i != headi:
                cur_label = arg_labels[i] if len(arg_labels) > i else label
                tuples, _ = gen_trigrams(arg, [head] + prev_heads, cur_label)
                out.extend(tuples)
        return out, head
        
gen_trigrams(ex)[0]

[('like_VV', [], 'root'),
 ('eat_V2', ['like_VV'], 'xcomp'),
 ('hay_N', ['eat_V2', 'like_VV'], 'dobj'),
 ('DefArt', ['hay_N', 'eat_V2', 'like_VV'], 'det'),
 ('NumSg', ['DefArt', 'hay_N', 'eat_V2', 'like_VV'], 'nummod'),
 ('horse_N', ['like_VV'], 'nsubj'),
 ('DefArt', ['horse_N', 'like_VV'], 'det'),
 ('NumSg', ['DefArt', 'horse_N', 'like_VV'], 'nummod'),
 ('TTAnt', ['like_VV'], 'empty'),
 ('PPos', ['like_VV'], 'empty'),
 ('which_IQuant', ['like_VV'], 'dep'),
 ('NumSg', ['which_IQuant', 'like_VV'], 'dep'),
 ('we_Pron', ['which_IQuant', 'like_VV'], 'dep'),
 ('all_Predet', ['we_Pron', 'which_IQuant', 'like_VV'], 'dep'),
 ('select_V', ['all_Predet', 'we_Pron', 'which_IQuant', 'like_VV'], 'dep'),
 ('TTAnt',
  ['select_V', 'all_Predet', 'we_Pron', 'which_IQuant', 'like_VV'],
  'empty'),
 ('PPos',
  ['select_V', 'all_Predet', 'we_Pron', 'which_IQuant', 'like_VV'],
  'empty')]

In [19]:
def find_head(expression):
    fun, args = expression.unpack()    
    head = labels[fun].index('head')
    if len(args) <= head: return fun
    return find_head(args[head])

find_head(ex)

'like_VV'

In [20]:
cat = gr.functionType('DefArt').cat
for ud_cat, gf_cats in parse.POSSIBLE_GF_CATS_BY_UD_CAT.items():
    if cat in gf_cats:
        print(ud_cat)

DET


In [14]:
['root']*2

['root', 'root']